In [1]:
import netCDF4 as nc
import xarray as xr
import numpy as np
from scipy.ndimage import filters
import os
import matplotlib.pyplot as plt
from matplotlib import gridspec
from copy import deepcopy
from shutil import copyfile
from tqdm import tqdm_notebook as tqdm
import logging

In [2]:
logger = logging.getLogger()
logging.basicConfig(level=logging.INFO)

rnd = np.random.RandomState(42)

In [3]:
DENSITY = 1000
EARTH_RADIUS = 6371000

n_ens_mems = 40

# Load data

In [4]:
RESTART_FILE = "/work/um0203/u300636/for2131/runs/da_enkf_for_soil/initial/orig/ens001/clm_ana20150730000000.nc"
OUTPUT_FILE = "/work/um0203/u300636/for2131/runs/da_enkf_for_soil/initial/orig/ens001/clmoas.clm2.h0.2015-07-30-00900.nc"
OUTPUT_FOLDER = "/work/um0203/u300636/for2131/runs/da_enkf_for_soil/initial"

In [5]:
restart_data = xr.open_dataset(RESTART_FILE)
output_data_raw = xr.open_dataset(OUTPUT_FILE)

In [6]:
output_data = output_data_raw.sel(time='2015-07-30T00:15:00').squeeze()
output_data = output_data.stack(stacked=['lat', 'lon'])

## Load H2O soil moisture

In [7]:
h2o_liq = restart_data['H2OSOI_LIQ'][:, 5:].copy(deep=True)
delta_z = output_data['DZSOI'].T.values
sat_point = output_data['WATSAT'].T.values
h2o_liq['column'] = output_data.indexes['stacked']

In [8]:
h2o_vol = h2o_liq / DENSITY / delta_z
wet_soi = h2o_vol / sat_point

In [9]:
h2o_vol.shape

(80634, 10)

In [9]:
unstacked_wet = wet_soi.unstack('column')

## Load soil temperature

In [10]:
soi_temp = restart_data['T_SOISNO'][:, 5:].copy(deep=True)
soi_temp['column'] = output_data.indexes['stacked']

In [11]:
unstacked_temp = soi_temp.unstack('column')

# Create perturbations

In [12]:
hori_sigma = 14
hori_trunc = 3

vert_sigma = 0.5
vert_trunc = 2

## Create vertical information

In [13]:
def gauss_func(dist, sigma=0.5, trunc=3):
    pre = 1 / sigma * np.sqrt(2*np.pi)
    gauss_val = pre * np.exp(-0.5*np.power((dist / sigma), 2))
    gauss_val[dist>sigma*trunc] = 0
    return gauss_val

In [14]:
depth = output_data.levsoi.values[:, None]

In [15]:
vert_dist = np.abs(depth - depth.T)
gauss_dist = gauss_func(vert_dist, vert_sigma, vert_trunc)
norm_gauss = (gauss_dist / gauss_dist.sum(axis=0)).T

## Soil moisture

In [16]:
soi_h2o_scale = 0.06

### Uncorrelated perturbation

In [17]:
uncorr_h2o_perts = rnd.normal(scale=soi_h2o_scale, size=(*unstacked_wet.shape, n_ens_mems+1))
uncorr_h2o_perts = uncorr_h2o_perts.transpose(3, 1, 2, 0)

### Horizontal correlation

In [18]:
hori_h2o_perts = filters.gaussian_filter(uncorr_h2o_perts, sigma=(0.0000001, hori_sigma, hori_sigma, 0.0000001), truncate=hori_trunc, mode='wrap')
hori_h2o_perts = hori_h2o_perts * soi_h2o_scale / hori_h2o_perts.std(axis=(0, -1))[None, ..., None]

### Vertical correlation

In [19]:
ens_h2o_perts = np.dot(hori_h2o_perts, norm_gauss.T)
ens_h2o_perts = ens_h2o_perts  * soi_h2o_scale / ens_h2o_perts.std(axis=(0, 1, 2))
ens_h2o_perts = ens_h2o_perts.transpose(0, 3, 1, 2)

In [20]:
print(ens_h2o_perts.std(axis=(0, 2, 3)))
print(ens_h2o_perts.std(axis=(0, 1)).mean())
print(ens_h2o_perts.std(axis=0).mean())

[0.06 0.06 0.06 0.06 0.06 0.06 0.06 0.06 0.06 0.06]
0.05938967240597581
0.058959365879258846


### Add perturbations + clipping + bias correction

In [21]:
perturbed_wet = np.clip(unstacked_wet.expand_dims('ensemble'), 0, 1) + ens_h2o_perts
perturbed_wet = np.clip(perturbed_wet, 0, 1)

In [22]:
perturbed_wet.std('ensemble').mean()

<xarray.DataArray 'H2OSOI_LIQ' ()>
array(0.05633672)

### To H2O perturbed value

In [23]:
perturbed_wet = perturbed_wet.stack(column=('lat', 'lon')).T
pert_h2o_liq = perturbed_wet * sat_point[..., None] * delta_z[..., None] * DENSITY

## Soil temperature

In [24]:
soi_temp_scale = 1.0

### Uncorrelated perturbation

In [25]:
uncorr_temp_perts = rnd.normal(scale=soi_temp_scale, size=(*unstacked_temp.shape, n_ens_mems+1))
uncorr_temp_perts = uncorr_temp_perts.transpose(3, 1, 2, 0)

### Horizontal correlation

In [26]:
hori_temp_perts = filters.gaussian_filter(uncorr_temp_perts, sigma=(0.0000001, hori_sigma, hori_sigma, 0.0000001), truncate=hori_trunc, mode='wrap')
hori_temp_perts = hori_temp_perts * soi_temp_scale / hori_temp_perts.std(axis=(0, -1))[None, ..., None]

### Vertical correlation

In [27]:
ens_temp_perts = np.dot(hori_temp_perts, norm_gauss.T)
ens_temp_perts = ens_temp_perts  * soi_temp_scale / ens_temp_perts.std(axis=(0, 1, 2))
ens_temp_perts = ens_temp_perts.transpose(0, 3, 1, 2)

In [28]:
print(ens_temp_perts.std(axis=(0, 2, 3)))
print(ens_temp_perts.std(axis=0).mean())
print(ens_temp_perts.std(axis=0).mean())

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
0.9828951625610811
0.9828951625610811


### Add perturbations

In [29]:
perturbed_temp = unstacked_temp.expand_dims('ensemble') + ens_temp_perts

### To temperature perturbed value

In [30]:
perturbed_temp = perturbed_temp.stack(column=('lat', 'lon')).T

# Write data

## ensemble data

In [31]:
for mem in tqdm(range(n_ens_mems)):
    path_mem = os.path.join(OUTPUT_FOLDER, 'ensemble_new', 'ens{0:03d}'.format(mem+1))
    if not os.path.exists(path_mem):
        os.makedirs(path_mem)
    out_file = os.path.join(path_mem, 'clm_ana20150730000000.nc')
    copyfile(RESTART_FILE, out_file)
    with nc.Dataset(out_file, mode='r+') as temp_ds:
        temp_ds['H2OSOI_LIQ'][:, 5:] = pert_h2o_liq[..., mem].values
        temp_ds['T_SOISNO'][:, 5:] = perturbed_temp[..., mem].values

/work/um0203/u300636/anaconda3/envs/pytassim-dev/lib/python3.7/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


## control run

In [32]:
out_file = os.path.join(OUTPUT_FOLDER, 'control_new', 'ens001', 'clm_ana20150730000000.nc')
if not os.path.exists(path_mem):
    os.makedirs(path_mem)
copyfile(RESTART_FILE, out_file)
with nc.Dataset(out_file, mode='r+') as temp_ds:
    temp_ds['H2OSOI_LIQ'][:, 5:] = pert_h2o_liq[..., -1].values
    temp_ds['T_SOISNO'][:, 5:] = perturbed_temp[..., -1].values

FileNotFoundError: [Errno 2] No such file or directory: '/work/um0203/u300636/for2131/runs/da_enkf_for_soil/initial/control_new/ens001/clm_ana20150730000000.nc'

## deterministic run

In [ ]:
pert_h2o_mean = pert_h2o_liq[..., :-1].mean(dim='ensemble')
pert_t2m_mean = perturbed_temp[..., :-1].mean(dim='ensemble')

In [37]:
out_file = os.path.join(OUTPUT_FOLDER, 'det_new', 'ens001', 'clm_ana20150730000000.nc')
copyfile(RESTART_FILE, out_file)
with nc.Dataset(out_file, mode='r+') as temp_ds:
    temp_ds['H2OSOI_LIQ'][:, 5:] = pert_h2o_mean.values
    temp_ds['T_SOISNO'][:, 5:] = pert_t2m_mean.values